In [27]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [28]:
years = list(range(2022, 2016, -1))
all_matches = []

In [29]:
years

[2022, 2021, 2020, 2019, 2018, 2017]

In [30]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [31]:
import time
for year in years:
    time.sleep(1)
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    time.sleep(1)
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    time.sleep(1)
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        time.sleep(1)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/keeper' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        keeper = pd.read_html(data.text, match="Goalkeeping")[0]
        keeper.columns = keeper.columns.droplevel()
        try:
            team_data = team_data.merge(keeper[["Date", "SoTA", "PKA", "PKsv", "PKm"]], on="Date")
        except ValueError:
            continue

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        passing = pd.read_html(data.text, match="Passing")[0]
        passing.columns = passing.columns.droplevel()
        try:
            passing.columns.values[10] = "Cmp_Total"
            passing.columns.values[11] = "Att_Total"    
            team_data = team_data.merge(passing[["Date", "Cmp_Total", "Att_Total", "TotDist", "PrgDist"]], on="Date")
        except ValueError:
            continue
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing_types' in l]      
        time.sleep(1)    
        data = requests.get(f"https://fbref.com{links[0]}")
        pass_types = pd.read_html(data.text, match="Pass Types")[0]
        pass_types.columns = pass_types.columns.droplevel()
        try:
            team_data = team_data.merge(pass_types[["Date", "TI", "CK"]], on="Date")
        except ValueError:
            continue

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/gca' in l]   
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        gca = pd.read_html(data.text, match="Goal and Shot Creation")[0] 
        gca.columns = gca.columns.droplevel()
        try:
            team_data = team_data.merge(gca[["Date", "SCA", "GCA"]], on="Date")
        except ValueError:
            continue

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/defense' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        defension_actions = pd.read_html(data.text, match="Defensive Actions")[0]
        defension_actions.columns = defension_actions.columns.droplevel()
        try:
            defension_actions.columns.values[10] = "Tkl_Defensive"
            defension_actions.columns.values[11] = "TklW_Defensive"
            defension_actions.columns.values[19] = "Blocks_Defensive"
            team_data = team_data.merge(defension_actions[["Date", "Tkl_Defensive", "TklW_Defensive", "Blocks_Defensive"]], on="Date")
        except ValueError:
            continue

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/possession' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        possession = pd.read_html(data.text, match="Possession")[0]
        possession.columns = possession.columns.droplevel()
        try:
            possession.columns.values[18] = "Dribbles_Succ"
            possession.columns.values[19] = "Dribles_Att"
            team_data = team_data.merge(possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Dribbles_Succ", "Dribles_Att", "Rec"]], on="Date")
        except ValueError:
            continue

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc' in l]        
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        msc = pd.read_html(data.text, match="Miscellaneous Stats")[0]
        msc.columns = msc.columns.droplevel()
        try:
            team_data = team_data.merge(msc[["Date", "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "PKwon", "PKcon", "OG"]], on="Date")
        except ValueError:
            continue


        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

IndexError: list index out of range

In [ ]:
len(all_matches)

20

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.to_csv('EPL_2017-2022.csv', mode='a', index=False, header=True)